In [1]:
# 트랜스포머 어텐션
# seq2seq

# 라이브러리
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import os
import io
import re
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [4]:
# 데이터셋 전처리를 위한 함수 정의
def unicode_to_ascii(s):        # 유니코드를 아스키로 변환
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')
 
def preprocess_sentence(w):     # 문장 전처리
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
# 전처리 내용 테스트
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))


<start> May I borrow this book ? <end>
b'<start> \xc2\xbf Puedo tomar prestado este libro ? <end>'


In [6]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
 
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
 
    return zip(*word_pairs)
def max_length(tensor):
    return max(len(t) for t in tensor)
 
def tokenize(lang):   # 문장의 토큰화
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)
 
  tensor = lang_tokenizer.texts_to_sequences(lang)
 
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
 
  return tensor, lang_tokenizer
 
def load_dataset(path, num_examples=None):    # 입력/출력(english, spanish) 쌍 만들기
    targ_lang, inp_lang = create_dataset(path, num_examples)
 
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
 
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [7]:
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset('./data/spa.txt', num_examples)
 
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
 
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [8]:
# 데이터 분석을 준비하기 앞서 하이퍼파라미터들을 정의
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
 
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
# 데이터 분석을 위한 데이터셋 준비
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [9]:
# 인코더 네트워크
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
 
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state
 
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))  # 은닉층 초기화
 
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) 

In [10]:
# 어텐션 구현
class EDAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(EDAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1) #1 텐서의 원하는 위치에 차원 추가
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis))) #2 배치 크기, 시퀀스 최대 길이 1
 
        attention_weights = tf.nn.softmax(score, axis=1) # 어텐션 가중치(attention_weight)의 형태는 (배치크기, 시퀸스 최대 길이, 1)이 됩니다.
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1) # 컨텍스트 벡터(context_vector)의 형태는 (배치크기, 은닉층 크기)입니다.
        return context_vector, attention_weights 
attention_layer = EDAttention(10)

In [11]:
# 디코더 네트워크
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = EDAttention(self.dec_units) #어텐션 적용
 
    def call(self, x, hidden, enc_output):  # 인코더 출력(enc_output)형태는 (배치크기, 시퀸스 최대 길이, 은닉층 크기)입니다.
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)  # 임베딩층을 통과한 후 x의 형태는 (배치크기, 1, 임베딩 차원)입니다.
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)  # 병합된 벡터를 GRU로 보냅니다.
        output = tf.reshape(output, (-1, output.shape[2])) # 출력 형태는 (배치크기x1, 은닉층 크기)입니다..
        x = self.fc(output)
        return x, state, attention_weights
 
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [12]:
# 모델 훈련을 위한 옵티마이저와 손실함수 정의
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
 
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [28]:
# 모델의 저장과 복원을 위한 체크포인트 설정
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder, epoch=tf.Variable(1))

# 체크포인트 로딩 코드 - 모델 훈련하는 함수 내에서 실행
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [29]:
# 모델을 훈련시키기 위한 함수 정의
def train_step(inp, targ, enc_hidden):
  loss = 0

# 체크포인트 로딩 코드
  # checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
  with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]): # 대상 단어를 입력으로 사용
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output) # 인코더 출력(enc_output)을 디코더로 보냄
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
  batch_loss = (loss / int(targ.shape[1]))  # 손실/오차 계산
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables)) 
  return batch_loss

In [30]:
# 모델 훈련
EPOCHS = 10
 
for epoch in range(EPOCHS):
  start = time.time()
 
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
 
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
 
    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  if (epoch + 1) % 2 == 0:  # 2 에포크마다 모델을 체크포인트에 저장
    checkpoint.save(file_prefix = checkpoint_prefix)
 
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))



Epoch 1 Batch 0 Loss 0.5423


KeyboardInterrupt: 

In [ ]:
# 모델 평가 함수
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
 
    sentence = preprocess_sentence(sentence)
 
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
 
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))  # 어텐션 가중치
        attention_plot[t] = attention_weights.numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        dec_input = tf.expand_dims([predicted_id], 0)    # 예측된 ID가 모델에 피드백
 
    return result, sentence, attention_plot

In [ ]:
# 어텐션 가중치 시각화를 위한 함수 정의
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
# 번역을 위한 함수 정의 및 번역 문장 입력 함수
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)
 
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
 
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' ')) #어텐션 가중치 매핑

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))  #1

translate(u'esta es mi vida.')   # 스페인어를 영어로 번역